In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

multi_lingual_sentiment_analysis_path = kagglehub.competition_download('multi-lingual-sentiment-analysis')
metaresearch_llama_3_1_transformers_8b_instruct_2_path = kagglehub.model_download('metaresearch/llama-3.1/Transformers/8b-instruct/2')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch transformers peft datasets accelerate sentencepiece

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer,BitsAndBytesConfig
from datasets import load_dataset,DatasetDict
from peft import LoraConfig, get_peft_model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

# print("Model loaded successfully in full precision!")
# print(10*'_',f"Base Model size: {model.get_memory_footprint():,} bytes\n")
# Apply 4-bit quantization

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Reload the model with quantization
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, low_cpu_mem_usage=True)

print("Model successfully quantized to 4-bit!")
print(10*'_',f"Base Model size: {model.get_memory_footprint():,} bytes\n")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model successfully quantized to 4-bit!
__________ Base Model size: 5,591,548,160 bytes



In [ ]:
# model.to("cuda")
model.to('cuda:0')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [ ]:
dataset=load_dataset('csv',data_files='/kaggle/input/multi-lingual-sentiment-analysis/train.csv')

# Split dataset (90% Train, 10% Validation)
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

# Rename splits for clarity
dataset = DatasetDict({
    "train": dataset["train"],
    "validation": dataset["test"]  # Treat this as validation
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['ID', 'sentence', 'label', 'language'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['ID', 'sentence', 'label', 'language'],
        num_rows: 100
    })
})


In [ ]:
def format_prompt(example):
    prompt = f"Classify the sentiment of the following language {example['language']} text: '{example['text']}' \n Sentiment:"
    return {"prompt": prompt, "labels": example["label"]}

tokenized_datasets = dataset.map(format_prompt)
tokenized_datasets = tokenized_datasets.remove_columns(["text", "id", "language"])
tokenized_datasets.set_format("torch")


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'sentence', 'label', 'language', 'prompt'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['ID', 'sentence', 'label', 'language', 'prompt'],
        num_rows: 100
    })
})

In [ ]:
dataset['train'][:1]['prompt']

["Below is a sentence in an Indian language. Classify its sentiment.\nSentence: 'ਮੈਂ ਬਹੁਤ ਲੰਬੇ ਸਮੇਂ ਤੋਂ ਲੈਕਮੇ ਉਤਪਾਦਾਂ ਦੀ ਵਰਤੋਂ ਕਰ ਰਿਹਾ/ਰਹੀ ਹਾਂ ਅਤੇ ਇਹ ਮੇਰੀ ਮੇਕ-ਅੱਪ ਕਿੱਟ ਵਿੱਚ ਹੁਣੇ ਆਇਆ ਹੈ ਜੋ ਇਸਦੀ ਬ੍ਰਾਂਡ ਇਮੇਜ ਅਨੁਸਾਰ ਖਰਾ ਉਤਰਦਾ ਹੈ। ਇਹ ਮੇਰੀ ਚਮੜੀ ਦੇ ਕਾਲੇ ਘੇਰਿਆਂ ਨੂੰ ਢੱਕਦਾ ਹੈ, ਇਸ ਨੂੰ ਚਮਕਦਾਰ ਬਣਾਉਂਦਾ ਹੈ ਅਤੇ ਇਕਸਾਰ ਟੋਨ ਦਿੰਦਾ।'\nSentiment:"]

In [ ]:
print("Available VRAM:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
print("Torch Reserved Memory:", torch.cuda.memory_reserved() / 1e9, "GB")
print("Torch Allocated Memory:", torch.cuda.memory_allocated() / 1e9, "GB")

Available VRAM: 15.828320256 GB
Torch Reserved Memory: 11.549016064 GB
Torch Allocated Memory: 11.435756032 GB


In [ ]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding is handled

In [ ]:
# LoRA Configuration
lora_config = LoraConfig(
    r=8,  # Low-rank dimension
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Prevents overfitting
    target_modules=["q_proj", "v_proj"],  # Optimizes attention layers
    # bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.gradient_checkpointing_enable()

# trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# total_params = sum(p.numel() for p in model.parameters())

# print(f"Trainable Parameters: {trainable_params:,}")
# print(f"Total Parameters: {total_params:,}")
# print(f"Percentage of Trainable Parameters: {100 * trainable_params / total_params:.4f}%")

In [ ]:
from transformers import DataCollatorForLanguageModeling

# Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples["prompt"], padding="max_length", truncation=True, max_length=256)

# Tokenize Dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Data Collator for Padding & Masking
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We are doing causal LM fine-tuning
)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Batch size for each GPU
    per_device_eval_batch_size=1,   # Batch size for evaluation
    gradient_accumulation_steps=8,  # Simulates batch_size=16
    evaluation_strategy="steps",    # Evaluate after each set of steps
    eval_steps=500,                 # Evaluate every 500 steps (adjust as needed)
    save_strategy="steps",         # Save model at each evaluation step
    save_steps=500,                # Save checkpoint every 500 steps
    save_total_limit=2,            # Keep only the 2 most recent checkpoints
    num_train_epochs=1,            # Number of epochs to train
    logging_dir="./logs",          # Log directory for training
    logging_steps=100,             # Log every 100 steps
    report_to="none",              # Disable reporting to other platforms
    fp16=True,                     # Enable mixed precision
    optim="adamw_torch",           # Optimizer choice
    lr_scheduler_type="cosine",    # Cosine decay for better convergence
    warmup_ratio=0.03,             # Warm-up ratio
    weight_decay=0.01,             # Regularization
    load_best_model_at_end=True,   # Load the best model based on eval loss
    push_to_hub=False              # Disable model uploading
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Check if GPU is available
print("CUDA Available:", torch.cuda.is_available())  # Should print True if GPU is available
print("Using Device:", torch.cuda.get_device_name(0))  # Should print the GPU model (A100, T4, etc.)


CUDA Available: True
Using Device: Tesla T4


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start Training with Step-wise Evaluation
trainer.train()
